# AWS Query on Cloud Stored Data

While data is necessary for all ML projects, the vast amount of datasets used can become too much for a local machine to store. For example, the 2 GB CSV file used in this example, read in locally has a noticable latency taking several seconds. An API would be an obvious solution as it solves many of the problems using a CSV presents. But for the sake of this example, as an individual long investor, I need an efficent way to store decades of price action that won't hinder my machine's preformance. 

Using data from [Kaggle](https://www.kaggle.com/tsaustin/us-historical-stock-prices-with-earnings-data), I've uploaded the csv to an AWS S3 Bucket in the cloud.

In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import tensorflow as tf
import boto3
from io import StringIO
import cufflinks as cf
import plotly.offline as plyo
plyo.init_notebook_mode(connected=True)

/Users/myles/anaconda3/lib/python3.7/site-packages/google/colab/data_table.py:30: UserWarning:

IPython.utils.traitlets has moved to a top-level traitlets package.



In [3]:
s3 = boto3.client('s3',region_name='us-east-1',aws_access_key_id=AccessKey, aws_secret_access_key = SecretKey)

In [4]:
exp = "SELECT * FROM S3Object s WHERE s._1 = 'AMZN';"

In [5]:
response = s3.select_object_content(Bucket='mylesnewbucket1',
                                   Key = 'stock_prices_latest.csv',
                                   ExpressionType='SQL',
                                   Expression = exp,
                                   InputSerialization = {'CSV': {'FileHeaderInfo':'IGNORE'}},
                                   OutputSerialization = {'CSV': {}})

In [6]:
records = []

for event in response['Payload']:
    if 'Records' in event:
        records.append(event['Records']['Payload'])
file_str = ''.join(r.decode('utf-8') for r in records)

select_df = pd.read_csv(StringIO(file_str), 
                   names=['symbol','date','open','high','low','close','close_adjusted','volume','split_coef'])
print(len(select_df))
print(select_df.head())

5547
  symbol        date    open     high       low   close  close_adjusted  \
0   AMZN  2016-04-11  596.14  604.000  594.9100  595.93          595.93   
1   AMZN  2009-03-31   72.61   74.500   72.1200   73.44           73.44   
2   AMZN  2011-05-18  194.13  198.283  193.2500  197.09          197.09   
3   AMZN  2015-11-05  647.10  657.000  643.0901  655.65          655.65   
4   AMZN  2009-07-28   83.84   85.640   82.6000   84.98           84.98   

    volume  split_coef  
0  2704267         1.0  
1  8918200         1.0  
2  4955800         1.0  
3  4723825         1.0  
4  8774900         1.0  


In [7]:
data = select_df

In [8]:
data

,symbol,date,open,high,low,close,close_adjusted,volume,split_coef
0,AMZN,2016-04-11,596.14,604.000,594.9100,595.93,595.93,2704267,1.0
1,AMZN,2009-03-31,72.61,74.500,72.1200,73.44,73.44,8918200,1.0
2,AMZN,2011-05-18,194.13,198.283,193.2500,197.09,197.09,4955800,1.0
3,AMZN,2015-11-05,647.10,657.000,643.0901,655.65,655.65,4723825,1.0
4,AMZN,2009-07-28,83.84,85.640,82.6000,84.98,84.98,8774900,1.0
...,...,...,...,...,...,...,...,...,...
5542,AMZN,2020-01-14,1885.88,1887.110,1858.5500,1869.44,1869.44,3446381,1.0
5543,AMZN,2020-01-17,1885.89,1886.640,1857.2500,1864.72,1864.72,3948459,1.0
5544,AMZN,2020-01-13,1891.31,1898.000,1880.8000,1891.30,1891.30,2753903,1.0
5545,AMZN,2020-01-15,1872.25,1878.860,1855.0900,1862.02,1862.02,2896592,1.0


In [9]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5547 entries, 0 to 5546
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   symbol          5547 non-null   object 
 1   date            5547 non-null   object 
 2   open            5547 non-null   float64
 3   high            5547 non-null   float64
 4   low             5547 non-null   float64
 5   close           5547 non-null   float64
 6   close_adjusted  5547 non-null   float64
 7   volume          5547 non-null   int64  
 8   split_coef      5547 non-null   float64
dtypes: float64(6), int64(1), object(2)
memory usage: 390.1+ KB


So, we've brought the memory usage down from 2 GB to 390 KB. One unfortunate aspect to this is that S3 does not support joins yet. So for example, this data also included two additional datasets for Dividends and Returns, if we wanted to incorporate those, we'd have to run another query and use Pandas' join method. 